# Hybrid Demucs from Colab

This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/)
This is only for separation with pre-trained models, not training!

You can either upload files manually (slow) or link your Google Drive account.

In [1]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-ho0ukhba/demucs_a5a48a0c4c0d476f818182a7df45da01
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-ho0ukhba/demucs_a5a48a0c4c0d476f818182a7df45da01
  Resolved https://github.com/facebookresearch/demucs to commit e976d93ecc3865e5757426930257e200846a520a
  Preparing metadata (setup.py) ... done


In [2]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
# two_stems = None   # only separate one stems from the rest, for instance
two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = 'dataset_cleaned/test' #path to the folder with all the mp3 files
out_path = 'dataset_cleaned_sep/test' #customize it

In [3]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


"""def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)
"""

"def from_upload():\n    out_path = Path('separated')\n    in_path = Path('tmp_in')\n\n    if in_path.exists():\n        rmtree(in_path)\n    in_path.mkdir()\n\n    if out_path.exists():\n        rmtree(out_path)\n    out_path.mkdir()\n\n    uploaded = files.upload()\n    for name, content in uploaded.items():\n        (in_path / name).write_bytes(content)\n    separate(in_path, out_path)\n"

In [4]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

Going to separate the files:
dataset_cleaned/test/chunk_13235.mp3
dataset_cleaned/test/chunk_13236.mp3
dataset_cleaned/test/chunk_13237.mp3
dataset_cleaned/test/chunk_13238.mp3
dataset_cleaned/test/chunk_13239.mp3
dataset_cleaned/test/chunk_13240.mp3
dataset_cleaned/test/chunk_13241.mp3
dataset_cleaned/test/chunk_13242.mp3
dataset_cleaned/test/chunk_13243.mp3
dataset_cleaned/test/chunk_13244.mp3
dataset_cleaned/test/chunk_13245.mp3
dataset_cleaned/test/chunk_13246.mp3
dataset_cleaned/test/chunk_13247.mp3
dataset_cleaned/test/chunk_13248.mp3
dataset_cleaned/test/chunk_13249.mp3
dataset_cleaned/test/chunk_13250.mp3
dataset_cleaned/test/chunk_13251.mp3
dataset_cleaned/test/chunk_13252.mp3
dataset_cleaned/test/chunk_13253.mp3
dataset_cleaned/test/chunk_13254.mp3
dataset_cleaned/test/chunk_13255.mp3
dataset_cleaned/test/chunk_13256.mp3
dataset_cleaned/test/chunk_14943.mp3
dataset_cleaned/test/chunk_14944.mp3
dataset_cleaned/test/chunk_14945.mp3
dataset_cleaned/test/chunk_14946.mp3
dataset_c

100%|██████████████████████████████████████████████████████████████████████████| 5.85/5.85 [00:00<00:00,  9.23seconds/s]


Separating track dataset_cleaned/test/chunk_13236.mp3


100%|██████████████████████████████████████████████████████████████████████████| 5.85/5.85 [00:00<00:00, 98.57seconds/s]


Separating track dataset_cleaned/test/chunk_13237.mp3


...

100%|██████████████████████████████████████████████████████████████████████████| 5.85/5.85 [00:00<00:00, 98.03seconds/s]


Separating track dataset_cleaned/test/chunk_14236.mp3


100%|█████████████████████████████████████████████████████████████████████████| 5.85/5.85 [00:00<00:00, 107.59seconds/s]


Separating track dataset_cleaned/test/chunk_14237.mp3


100%|█████████████████████████████████████████████████████████████████████████| 5.85/5.85 [00:00<00:00, 111.40seconds/s]


In [5]:
# execute this to move all the mp3 into one folder
import os
dir = os.path.join(out_path,'htdemucs')
new_dir = out_path
for d in os.listdir(dir):
  os.rename(os.path.join(dir,d,'vocals.mp3'), os.path.join(new_dir,f'{d}.mp3'))
  """"""

In [6]:
"""# no need to execute
# This is manual upload and download :)
from_upload()
!zip -r separated.zip separated
files.download('./separated.zip')"""

"# no need to execute\n# This is manual upload and download :)\nfrom_upload()\n!zip -r separated.zip separated\nfiles.download('./separated.zip')"